In [ ]:
# default_exp stats

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# stats

> A way to access metadata on all the files due for processing.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import os
from pathlib import Path
from typing import Any
import fitz

from rich import print
from rich.console import Console
from rich.table import Table

In [ ]:
%load_ext rich

The rich extension is already loaded. To reload it, use:
  %reload_ext rich


In [ ]:
#export
def get_page_count(filepath: "os.PathLike[Any]") -> int:
    """Gets the page count of a PDF file."""
    pdf_obj = fitz.open(filepath)
    return pdf_obj.page_count

In [ ]:
#export
def add_comma_separation(input: int) -> str:
    """Adds comma-separation for thousands to an integer."""
    return f'{input:,.0f}'

In [ ]:
#export
def get_stats(source_path: "os.PathLike[Any]") -> int:
    """Displays statistics on the PDF data contained in a particular repository."""
    table = Table(title="Stats for your PDF Files")

    table.add_column("PageCount", justify="right", style="green")
    table.add_column("Filename", justify="left", style="cyan", no_wrap=True, max_width=50)


    count = 0
    file_number = 0
    files = list(source_path.glob("**/*.pdf")) # searches source_path and all subfolders
    for file in files:
        file_number += 1
        current_file_count = get_page_count(file)
        count += current_file_count
        if file_number <= 45:
            table.add_row(str(current_file_count), file.name)

    if file_number > 45:
        table.add_row("...", "...")
        
    console = Console()
    console.print(table)
    
    bold_str_count = f"[bold red]{add_comma_separation(count)}"
    console.print("[bold red]TOTAL PAGECOUNT:", bold_str_count)

In [ ]:
source = Path("/Users/strickvl/Desktop/NL")

get_stats(source)

                     Stats for your PDF Files                     
┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ PageCount ┃ Filename                                           ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         1 │ Dutch Friends of importance to me.pdf              │
│        18 │ 2021-10-21-mama-financial-overview-van-lanschot.p… │
│         2 │ Tax Call Mazars_25Oct2021.pdf                      │
│        18 │ 2021-10-21-mama-financial-overview-van-lanschot-C… │
│         1 │ medical exemption westeinde.pdf                    │
│         1 │ saba-invitation-letter.pdf                         │
│         2 │ 2021-10-21-mama-executor-instructions.pdf          │
│         6 │ notaris-conversation.pdf                           │
│         2 │ FINAL Family Emergency Contact List-20_05_20.pdf   │
│         1 │ bieslandhof-palliative.pdf                         │
│        13 │ 2021-10-21-mama-tax-return-2020.pdf                │
│         1 │ 20210817132930.pdf                                 │
│        13 │ 2021-10-21-mama-tax-return-2020-COMPRESSED.pdf     │
│         3 │ WILSVERKLARING + FINAL-SIGNED-AS.pdf               │
│         3 │ 193158 certified final translation.pdf             │
│         1 │ 12-11 Phase 2.pdf                                  │
│         1 │ 12-11 Phase 3.pdf                                  │
│         1 │ 12-11 Phase 1.pdf                                  │
│         1 │ 12-11 Offer introduction.pdf                       │
│         1 │ Hour indication.pdf                                │
│         1 │ 12-11 Phase 4.pdf                                  │
│         1 │ Order Confirmation Phase 1.pdf                     │
│         3 │ Privacy statement ENG 4-2-2021 DEF.pdf             │
│         1 │ Financial offer.pdf                                │
│         4 │ Terms and conditions Jeltje ENG 4-2-2021 DEF.pdf   │
│         1 │ Project description Phase 3.pdf                    │
│       340 │ NN-werkboek-2015-herzien-omslag-lowres.pdf         │
│        28 │ ENGELS_Handleiding Naar Nederland.pdf              │
│       112 │ NN-fotoboek-2014-herzien-omslag-lowres.pdf         │
└───────────┴────────────────────────────────────────────────────┘

TOTAL PAGECOUNT: 581